#  分子的无监督嵌入学习

在本教程中，我们将使用 `SeqToSeq` 模型来生成分子指纹来对分子进行分类。 主要参考下面这篇文献："Seq2seq Fingerprint: An Unsupervised Deep Molecular Embedding for Drug Discovery"（https://doi.org/10.1145/3107411.3107424）。

## Colab

本教程和目录中的其余部分都是在 Google colab 中完成。如果您想在 colab 中打开此笔记本，您可以点击以下链接。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioGavin/deepchem/blob/master/examples/tutorials-zh/Learning_Unsupervised_Embeddings_for_Molecules.ipynb)


In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

# 用SeqToSeq学习嵌入

许多类型的模型要求它们的输入有一个固定的形状。 因为不同分子的原子和键的数量上可以有很大的差异，这使得这些模型很难具有普适性。 我们需要一种为每个分子生成固定长度 "指纹 "的方法。 各种各样的方法已经被设计出来了，例如我们之前教程中使用的扩展连接性指纹（ECFPs）。 但是在这个例子中，我们将让 `SeqToSeq` 模型学习自己的方法来创建指纹，而不是人工设计指纹。

一个 `SeqToSeq` 模型执行序列到序列的转换。 例如，它们经常被用来将文本从一种语言翻译成另一种语言。 它由两部分组成，分别为 "编码器 "和 "解码器"。 编码器是一叠递归层。 序列输入时，一次一个符号，它生成一个固定长度的矢量，称为 "嵌入矢量"。 解码器是另一个递归层的堆叠，执行相反的操作：它将嵌入向量作为输入，并生成输出序列。 通过适当选择输入/输出进行训练，你可以创建一个模型，执行许多种类的转换。

在这种情况下，我们将使用描述分子的 SMILES 字符串作为输入序列。 我们将把模型训练成一个自动编码器，所以它试图使得输出序列与输入序列保持一致。 要做到这一点，编码器必须创建包含原始序列所有信息的嵌入向量。 这正是我们在指纹中想要的，也许这些嵌入向量随后会在其他模型中用来表示分子！

让我们来加载数据。 我们将使用 MUV 数据集。 它的训练集包括74501个分子，验证集包括9313个分子，所以它给我们提供了大量的 SMILES 字符串来处理。

In [1]:
import deepchem as dc
tasks, datasets, transformers = dc.molnet.load_muv(split='stratified')
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids

我们需要为我们的 `SeqToSeq`  模型定义 "规则"，即可以出现在序列中的所有标记。 (输入和输出序列也可以有不同的规则，但因为我们是把它当作一个自动编码器来训练的，在此情况下输入和输出的规则是相同的)。 把出现在任何训练序列中的每个字符列出来。

In [2]:
tokens = set()
for s in train_smiles:
  tokens = tokens.union(set(c for c in s))
tokens = sorted(list(tokens))

创建模型并定义要使用的优化方法。 在这种情况下，如果我们逐渐降低学习率，学习效果会好很多。 我们使用 `ExponentialDecay` 在每次遍历后将学习率乘以0.9。

In [3]:
from deepchem.models.optimizers import Adam, ExponentialDecay
max_length = max(len(s) for s in train_smiles)
batch_size = 100
batches_per_epoch = len(train_smiles)/batch_size
model = dc.models.SeqToSeq(tokens,
                           tokens,
                           max_length,
                           encoder_layers=2,
                           decoder_layers=2,
                           embedding_dimension=256,
                           model_dir='fingerprint',
                           batch_size=batch_size,
                           learning_rate=ExponentialDecay(0.001, 0.9, batches_per_epoch))

然后开始训练!  `fit_sequences()`的输入是一个生成器，它产生成对的输入/输出。 在一个好的GPU上，这应该需要几个小时或更少。

In [4]:
def generate_sequences(epochs):
  for i in range(epochs):
    for s in train_smiles:
      yield (s, s)

model.fit_sequences(generate_sequences(40))

让我们看看它作为一个自动编码器的效果如何。 我们将通过它运行验证集中的前500个分子，看看其中有多少是完全可重复的。

In [5]:
predicted = model.predict_from_sequences(valid_smiles[:500])
count = 0
for s,p in zip(valid_smiles[:500], predicted):
  if ''.join(p) == s:
    count += 1
print('reproduced', count, 'of 500 validation SMILES strings')

reproduced 161 of 500 validation SMILES strings


现在我们将尝试使用这个编码器来生成分子指纹。 我们计算训练和验证数据集中所有分子的嵌入向量，并创建新的数据集，把这些作为其特征向量。 数据量很小，我们可以直接将所有的东西存储在内存中。

In [6]:
import numpy as np
train_embeddings = model.predict_embeddings(train_smiles)
train_embeddings_dataset = dc.data.NumpyDataset(train_embeddings,
                                                train_dataset.y,
                                                train_dataset.w.astype(np.float32),
                                                train_dataset.ids)

valid_embeddings = model.predict_embeddings(valid_smiles)
valid_embeddings_dataset = dc.data.NumpyDataset(valid_embeddings,
                                                valid_dataset.y,
                                                valid_dataset.w.astype(np.float32),
                                                valid_dataset.ids)

我们将使用一个简单的、有一个隐藏层的全连接网络来进行分类。

In [7]:
classifier = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                                                      n_features=256,
                                                      layer_sizes=[512])
classifier.fit(train_embeddings_dataset, nb_epoch=10)

0.0014195525646209716

为了验证它的效果如何。我们分别计算训练和验证数据集的 ROC 和 AUC 。

In [8]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean, mode="classification")
train_score = classifier.evaluate(train_embeddings_dataset, [metric], transformers)
valid_score = classifier.evaluate(valid_embeddings_dataset, [metric], transformers)
print('Training set ROC AUC:', train_score)
print('Validation set ROC AUC:', valid_score)

Training set ROC AUC: {'mean-roc_auc_score': 0.9598792603154332}
Validation set ROC AUC: {'mean-roc_auc_score': 0.7251350862464794}


# 恭喜！是时候加入社区了！

恭喜您完成本教程笔记本！如果您喜欢本教程并希望继续使用 DeepChem，我们鼓励您完成本系列的其余教程。您还可以通过以下方式帮助 DeepChem 社区：

## 在 [GitHub](https://github.com/deepchem/deepchem) 上为 DeepChem 点亮小星星
这有助于大家建立对 DeepChem 项目和我们正在尝试构建的开源药物发现工具的共识。

## 加入 DeepChem Gitter
DeepChem [Gitter](https://gitter.im/deepchem/Lobby) 聚集了许多对生命科学深度学习感兴趣的科学家、开发人员和爱好者，欢迎加入！